In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

C:\Users\user\miniconda3\envs\ml\lib\site-packages\requests\__init__.py:78: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({0}) or chardet ({1}) doesn't match a supported "


In [2]:
DEVICE = 'cpu'

In [3]:
BATCH_SIZE = 32
EPOCHS = 10

In [4]:
train_dataset = datasets.CIFAR10(root = "../data/CIFAR10",
                                      train = True,
                                      download = True,
                                      transform = transforms.ToTensor())

test_dataset = datasets.CIFAR10(root = "../data/CIFAR10",
                                     train = False,
                                     transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

Files already downloaded and verified


In [ ]:
train,target =  train_dataset[0]
print(train.shape,  target)
np.transpose(train,(1,2,0)).shape

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
# train 데이터를 10개만 가져오자
for cnt, (x,y) in enumerate(train_dataset):
    if cnt >= 10: break 
    plt.subplot(1,10,cnt+1)
    plt.axis('off')
    # TypeError: Invalid shape (3, 32, 32) for image data
    plt.imshow(np.transpose(x, (1,2,0)))    
    plt.title(y)




In [ ]:
# 분류 10 설계  MLP
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,10)
    def forward(self,x):
        x = x.view(-1,32*32*3)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1)
        return x      


In [ ]:
model = Net().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss()
print(model)

In [5]:
# model.train()  전체 레이어를 사용 : 전체 layer의 전원을 켠다
# model.eval() evaluation, 사용하지 않는 layer들의 전원을 끈다
from tqdm import tqdm # 진행상황을 progress bar로 표시
def train(model, train_loader,optimizer):
    model.train()
    for image, label in tqdm(train_loader):  # 배치단위로 출력
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)        
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

In [6]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1,keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.*correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
for epoch in range(1, EPOCHS+1):
    train(model,train_loader,optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print(f"epoch : {epoch}  loss : {test_loss}, accuracy : {test_accuracy}")

In [7]:
# CNN 모델 설계
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels = 3, out_channels = 8,
            kernel_size = 3,
            padding = 1
        )
        self.conv2 = nn.Conv2d(
            in_channels = 8, out_channels = 16,
            kernel_size = 3,
            padding = 1
        )
        self.pool = nn.MaxPool2d(
            kernel_size =2, stride = 2
        )

        self.fc1 = nn.Linear(8 * 8 * 16, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)
  
    def forward(self, x):
        # x = (32, 32, 3)        
        x = self.conv1(x) # x = (32, 32, 3) ## len(x) - kernel_size + 1        
        x = F.relu(x)
        x = self.pool(x) # x = (16, 16, 3)        
        x = self.conv2(x) # x = (16, 16, 3)         
        x = F.relu(x)
        x = self.pool(x) # x = (8, 8, 3)        

        x = x.view(-1, 8* 8 * 16)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x,dim=1)

        return x
        
        

In [ ]:
cnnModel = CNN().to(DEVICE)
optimizer = torch.optim.Adam(cnnModel.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss()
for epoch in range(1, EPOCHS+1):
    train(cnnModel,train_loader,optimizer)
    test_loss, test_accuracy = evaluate(cnnModel, test_loader)
    print(f"epoch : {epoch}  loss : {test_loss}, accuracy : {test_accuracy}")    

100%|███████████████████████████████████████████████████████████████| 1563/1563 [00:26<00:00, 59.98it/s]


epoch : 1  loss : 0.06532504324913024, accuracy : 36.58


100%|███████████████████████████████████████████████████████████████| 1563/1563 [00:26<00:00, 57.92it/s]


epoch : 2  loss : 0.0637384945988655, accuracy : 42.23


100%|███████████████████████████████████████████████████████████████| 1563/1563 [00:27<00:00, 57.62it/s]


...

100%|███████████████████████████████████████████████████████████████| 1563/1563 [00:27<00:00, 57.46it/s]


epoch : 4  loss : 0.0619778923034668, accuracy : 47.82


100%|███████████████████████████████████████████████████████████████| 1563/1563 [00:30<00:00, 51.91it/s]


epoch : 5  loss : 0.06147929986715317, accuracy : 49.56


 69%|███████████████████████████████████████████▎                   | 1076/1563 [00:21<00:10, 48.37it/s]